<a href="https://colab.research.google.com/github/Mohamed-S-Helal/Arabic-Diwani-OCR/blob/main/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Get data from drive
# !cp -r /content/drive/MyDrive/data .
# !cp -r /content/drive/MyDrive/data_combined .

^C


In [3]:
!pip install -r requirements.txt -q

In [4]:
import PIL
from PIL import ImageFont
from PIL import Image as im
from PIL import ImageDraw
import io
import os
import arabic_reshaper
from bidi.algorithm import get_display
import numpy as np
from google.colab.patches import cv2_imshow
import cv2 as cv
import time
from tqdm import tqdm
from glob import glob
import pickle
import multiprocessing as mp
import re
import random
from sklearn.utils import shuffle
from sklearn.model_selection  import train_test_split
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from skimage.morphology import skeletonize, thin
from scipy.ndimage import interpolation as inter
import shutil

In [5]:
dir = os.getcwd()
print('current directory:', dir)

current directory: /content


In [6]:
def projection(gray_img, axis:str='horizontal'):
    """ Compute the horizontal or the vertical projection of a gray image """

    if axis == 'horizontal':
        projection_bins = np.sum(gray_img, 1).astype('int32')
    elif axis == 'vertical':
        projection_bins = np.sum(gray_img, 0).astype('int32')

    return projection_bins

def bound_box(img_char):
    HP = projection(img_char, 'horizontal')
    VP = projection(img_char, 'vertical')

    top = -1
    down = -1
    left = -1
    right = -1

    i = 0
    while i < len(HP):
        if HP[i] != 0:
            top = i
            break
        i += 1

    i = len(HP)-1
    while i >= 0:
        if HP[i] != 0:
            down = i
            break
        i -= 1

    i = 0
    while i < len(VP):
        if VP[i] != 0:
            left = i
            break
        i += 1

    i = len(VP)-1
    while i >= 0:
        if VP[i] != 0:
            right = i
            break
        i -= 1

    return img_char[top:down+1, left:right+1]


def binarize(char_img):
    if char_img is None:
      return
    _, binary_img = cv.threshold(char_img, 0, 255, cv.THRESH_BINARY)
    # _, binary_img = cv.threshold(char_img, 127, 255, cv.THRESH_BINARY)
    # _, binary_img = cv.threshold(word_img, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
    
    binary_char = binary_img // 255

    return binary_char

def featurizer(char_img):

    flat_char = char_img.flatten()

    return flat_char

In [10]:
chars = ['ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف',
'ق','ك', 'ل', 'م', 'ن', 'ه', 'و','ي','لا']
train_ratio = .9
script_path = os.getcwd()
classifiers = [MLPClassifier(alpha=1e-4, hidden_layer_sizes=(100,), max_iter=1000),
              MLPClassifier(alpha=1e-5, hidden_layer_sizes=(200, 100,), max_iter=1000),
              GaussianNB()]

names = ['1L_NN', '2L_NN', 'Gaussian_Naive_Bayes']
skip = [0, 0, 0]

width = 40
height = 40
dim = (width, height)

In [11]:
def prepare_char(char_img):
    
    binary_char = binarize(char_img)

    try:
        char_box = bound_box(binary_char)
        resized = cv.resize(char_box, dim, interpolation = cv.INTER_AREA)
        # print(resized)
    except:
        return

    return resized

def read_data(path, limit=4000):
    X = []
    Y = []
    print("For each char")
    for char in tqdm(chars, total=len(chars)):
        folder = f'{path}/{char}/'
        char_paths =  glob(f'{path}/{char}/*.png')
        # char_paths =  glob(f'data_combined/{char}/*.png')
        # print(os.path.exists(folder))
        # return
        if os.path.exists(folder):
            print(f'\nReading images for char {char}')
            for char_path in tqdm(char_paths[:limit], total=len(char_paths)):
                num = re.findall(r'\d+', char_path)[0]
                char_img = cv.imread(f'{folder}/{num}.png', 0)
                ready_char = prepare_char(char_img)
                if ready_char is None:
                  continue
                feature_vector = featurizer(ready_char)
                X.append(feature_vector)
                Y.append(char)

            os.chdir(script_path)

    return X+X, Y+Y

In [12]:
def train(data, destination):
    X, Y = read_data(data)

    assert(len(X) == len(Y))

    X, Y = shuffle(X, Y)

    X_train = []
    Y_train = []
    X_test = []
    Y_test = []

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=.9)
    
    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
    X_test = np.array(X_test)
    Y_test = np.array(Y_test)

    scores = []
    for idx, clf in tqdm(enumerate(classifiers), desc='Classifiers'):

        if not skip[idx]:

            clf.fit(X_train, Y_train)
            score = clf.score(X_test, Y_test)
            scores.append(score)
            print(score)

            # Save the model
            # destination = f'models'
            if not os.path.exists(destination):
                os.makedirs(destination)
            
            location = f'{destination}/{names[idx]}.sav'
            pickle.dump(clf, open(location, 'wb'))


    with open(f'{destination}/report.txt', 'w') as fo:
        for score, name in zip(scores, names):
            fo.writelines(f'Score of {name}: {score}\n')


In [15]:
def duplicate_data(folder):
  for i in os.listdir(folder):
    for j in os.listdir(f'{folder}/{i}'):
      path = f'{folder}/{i}'
      shutil.copyfile(f'{path}/{j}',f'{path}/0000000{j}')

In [16]:
duplicate_data('data')
# duplicate_data('data_combined')

In [17]:
## train using only generated data
train('data', 'models1')

100%|██████████| 16/16 [00:00<00:00, 3836.33it/s]

100%|██████████| 6/6 [00:00<00:00, 3749.94it/s]

100%|██████████| 6/6 [00:00<00:00, 3895.04it/s]

100%|██████████| 6/6 [00:00<00:00, 3806.08it/s]

100%|██████████| 8/8 [00:00<00:00, 3932.77it/s]

100%|██████████| 8/8 [00:00<00:00, 4192.73it/s]

100%|██████████| 8/8 [00:00<00:00, 4207.45it/s]

100%|██████████| 4/4 [00:00<00:00, 3498.90it/s]

100%|██████████| 4/4 [00:00<00:00, 3518.71it/s]

100%|██████████| 4/4 [00:00<00:00, 3611.11it/s]

100%|██████████| 4/4 [00:00<00:00, 3291.59it/s]

100%|██████████| 8/8 [00:00<00:00, 4222.28it/s]

100%|██████████| 8/8 [00:00<00:00, 4051.49it/s]

100%|██████████| 8/8 [00:00<00:00, 4288.11it/s]

100%|██████████| 8/8 [00:00<00:00, 4193.26it/s]

100%|██████████| 8/8 [00:00<00:00, 4221.75it/s]

100%|██████████| 8/8 [00:00<00:00, 4234.00it/s]

100%|██████████| 8/8 [00:00<00:00, 4238.28it/s]

100%|██████████| 8/8 [00:00<00:00, 4034.44it/s]

100%|██████████| 8/8 [00:00<00:00, 2073.18it/s]

100%|██████████| 8

For each char

Reading images for char ا

Reading images for char ب

Reading images for char ت

Reading images for char ث

Reading images for char ج

Reading images for char ح

Reading images for char خ

Reading images for char د

Reading images for char ذ

Reading images for char ر

Reading images for char ز

Reading images for char س

Reading images for char ش

Reading images for char ص

Reading images for char ض

Reading images for char ط

Reading images for char ظ

Reading images for char ع

Reading images for char غ

Reading images for char ف

Reading images for char ق

Reading images for char ك

Reading images for char ل

Reading images for char م


100%|██████████| 8/8 [00:00<00:00, 4323.47it/s]

100%|██████████| 8/8 [00:00<00:00, 1302.48it/s]

100%|██████████| 8/8 [00:00<00:00, 4271.19it/s]

100%|██████████| 10/10 [00:00<00:00, 4989.06it/s]

100%|██████████| 29/29 [00:00<00:00, 112.92it/s]
Classifiers: 0it [00:00, ?it/s]


Reading images for char ن

Reading images for char ه

Reading images for char و

Reading images for char ي

Reading images for char لا


Classifiers: 1it [00:03,  3.04s/it]

1.0


Classifiers: 3it [00:06,  2.32s/it]

1.0
1.0


In [18]:
## train using combinrd data generated + online dataset
train('data_combined', 'models2')

  2%|▏         | 102/4213 [00:00<00:04, 1016.34it/s]

For each char

Reading images for char ا



  2%|▏         | 90/4109 [00:00<00:04, 898.59it/s]


Reading images for char ب



  2%|▏         | 84/4276 [00:00<00:05, 833.92it/s]

...

0.9987068965517242


Classifiers: 2it [34:16, 1101.57s/it]

0.999353448275862


Classifiers: 3it [34:24, 688.32s/it]

0.8703879310344828


In [19]:
!cp -r models1 /content/drive/MyDrive
!cp -r models2 /content/drive/MyDrive